In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
from random import randint
from copy import deepcopy
import pickle as pkl
import argparse

from torch import nn
from models import AutoEncoder, AutoEncoderSimple, load_model
import torch.nn.functional as F

sys.path.append('../../src')
sys.path.append('../../src/vae')
sys.path.append('../../src/vae/models')
sys.path.append('../../src/dsets/cosmology')
from dset import get_dataloader
from model import init_specific_model
from losses import get_loss_f, _reconstruction_loss
from training import Trainer
from viz import viz_im_r, cshow, viz_filters
from sim_cosmology import p

sys.path.append('../../lib/trim')
# trim modules
from trim import DecoderEncoder

## VAE on cosmology data

In [2]:
# create dir
out_dir = opj(p.out_dir, p.dirname)
os.makedirs(out_dir, exist_ok=True)  

# seed
random.seed(p.seed)
np.random.seed(p.seed)
torch.manual_seed(p.seed)

# get dataloaders
train_loader = get_dataloader(p.data_path, 
                              batch_size=p.train_batch_size)
im = iter(train_loader).next()[0].to(device)

# prepare model
model = AutoEncoder(img_size=p.img_size).to(device)

# train
optimizer = torch.optim.Adam(model.parameters(), lr=p.lr)

In [3]:
# Training Loop
# Lists to keep track of progress
losses = []
num_epochs = 100

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    epoch_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader, 0):
        data = data.to(device)
        recon_data, latent_sample = model(data)
        # loss
        loss = _reconstruction_loss(data, recon_data, distribution="gaussian", storer=None)
        # zero grad
        optimizer.zero_grad()
        # backward
        loss.backward()
        # Update step
        optimizer.step()
        
        epoch_loss += loss.data.item()

        # Output training stats
        if batch_idx % 50 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data.item()), end='')
        

    # Save Losses for plotting later
    losses.append(epoch_loss/(batch_idx + 1))

# SAVE MODEL
# torch.save(model.state_dict(), 'results/autoenc_2layer.pth')

Starting Training Loop...
Train Epoch: 99 [28800/30000 (96%)]	Loss: 0.137391

In [6]:
im = iter(train_loader).next()[0].to(device)
recon_data, latent_sample = model(im)
viz_im_r(im[0,0], recon_data[0,0])

AttributeError: 'AutoEncoder' object has no attribute 'conv3'

In [ ]:
# loss versus training iterations
plt.figure(figsize=(10,5))
plt.title("Loss During Training")
plt.plot(np.log(losses), label="epoch-loss")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
latent_sample.shape

In [ ]:
model.conv2.weight.shape

In [ ]:
viz_filters(model.conv2.weight[:,1:2,...], n_row=1, n_col=3, resize_fac=3, normalize=True, title='Conv Filters')

In [ ]:
viz_filters(model.convT2.weight, n_row=1, n_col=3, resize_fac=3, normalize=True, title='Transpose Conv Filters')

In [ ]:
model.encoder(im)[0].shape

In [ ]:
viz_filters(model.encoder(im)[0], n_row=2, n_col=3, normalize=True, resize_fac=1, title='Representation')